In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import time
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

from models import tiramisu
from datasets import ivy_gap


In [33]:
dataset = ivy_gap.IvyGap(
    root="Ivy_gap_dataset", split="train")

In [37]:
raw, annot = dataset[5]
annot.show()